# Captcha Recognition With K-Nearest-Neighbours

Import required libraries

In [ ]:
import numpy as np                  # used for creating numpy arrays for model creation
import cv2                          # used for image processing
import os                           # used to read files and directory structure

Define global variables

In [ ]:
RESIZED_IMAGE_WIDTH = 20                    # width of resized image
RESIZED_IMAGE_HEIGHT = 30                   # height of resized image

DATASET_PATH = os.getcwd() + '/dataset/'    # Dataset path

## TRAIN MODEL

### Load Dataset

Open training characters image

In [ ]:
X = np.empty((0, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT), 
            dtype = np.float32)         # flattened images
y = []                                  # labels

In [ ]:
for char in os.listdir(DATASET_PATH):               # for each character folder
    print(char)                                     # print the character name
    if (len(char) > 1):                             # if the folder name is more than 1 character, skip it
        continue
    for img in os.listdir(DATASET_PATH + char):     # for each image in the folder
        if img[-3:] != 'png':                       # if the image is not a png file, skip it
            continue
        image = cv2.imread(DATASET_PATH + char + '/' + img, 
                            cv2.IMREAD_GRAYSCALE)   # read the image
        flattenedImage = image.reshape((1, 
                    RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))    # flatten the image
        X = np.append(X, flattenedImage, 0)         # append the image to the X array
        y.append(ord(char))                         # append the class number to the y array

In [ ]:
# convert to numpy arrays
y = np.array(y, np.float32)

In [ ]:
print(X.shape)
print(y.shape)

Plot Dataset

In [ ]:
import seaborn as sns                   # used for plotting graphs
import matplotlib.pyplot as plt         # used for plotting graphs

In [ ]:
plot = np.unique(y, return_counts=True)                 # count the number of times each class occurs
for i in range(len(plot[0])):
    print(chr(int(plot[0][i])), plot[1][i])             # print the character and its count

In [ ]:
plt.figure(figsize = (30, 7))                                       # set the size of the plot
sns.barplot(x = plot[0], y = plot[1], palette='Blues_d')            # plot the graph
plt.show()                                                          # show the graph

### Split dataset in training and testing data

Import Libraries

In [ ]:
from sklearn.neighbors import KNeighborsClassifier       # KNN classifier
from sklearn.model_selection import train_test_split     # used for splitting the dataset into training and testing sets
from sklearn.metrics import classification_report        # used for evaluating the model

Split Dataset

In [ ]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

Plot Training Data

In [ ]:
plot_train = np.unique(y_train, return_counts=True)     # count the number of times each class occurs in training set
for i in range(len(plot_train[0])):
    print(chr(int(plot_train[0][i])), plot_train[1][i]) # print the character and its count

In [ ]:
plt.figure(figsize = (30, 7))                                   # set the size of the plot
sns.barplot(x = plot_train[0], 
            y = plot_train[1], palette='Blues_d')               # plot the graph
plt.show()                                                      # show the graph

Plot Testing Data

In [ ]:
plot_test = np.unique(y_test, return_counts=True)       # count the number of times each class occurs in testing set
for i in range(len(plot_test[0])):  
    print(chr(int(plot_test[0][i])), plot_test[1][i])   # print the character and its count

In [ ]:
plt.figure(figsize = (30, 7))                                       # set the size of the plot
sns.barplot(x = plot_test[0], y = plot_test[1], palette='Blues_d')  # plot the graph
plt.show()                                                          # show the graph

### Create K-Nearest-Neighbours Model

Create KNN object

In [ ]:
kNearest = KNeighborsClassifier(n_neighbors = 3)    # create the KNN classifier
kNearest.fit(X_train, y_train)                      # train the model

Print Model Statistics

In [ ]:
print(kNearest.score(X_test, y_test))                           # print accuracy
print(classification_report(y_test, kNearest.predict(X_test)))  # print classification report

## TEST MODEL

In [ ]:
TEST_IMAGE = 'test_images/' + '2LKGD.png'       # Test image path

Open testing image

In [ ]:
img = cv2.imread(TEST_IMAGE)                # read in testing image
if img is None:                             # if image was not read successfully, exit program
    print("error: image not read from file \n\n")
    exit(0)

Preprocess Image

In [ ]:
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                             # convert to grayscale
# cv2.imshow('gray', gray)
# cv2.waitKey(0)

thresh = cv2.adaptiveThreshold( 
                imgGray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                cv2.THRESH_BINARY, 11, 0)                                   # get binary image
# cv2.imshow('thresh', thresh)
# cv2.waitKey(0)

close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE,
                         np.ones((3, 3), np.uint8))                         # remove noise from image
# cv2.imshow('close', close)
# cv2.waitKey(0)

dilate = cv2.dilate(close, np.ones((2, 2), np.uint8),
                    iterations=1)                                           # get dilated image
# cv2.imshow('dilate', dilate)
# cv2.waitKey(0)

image = cv2.bitwise_not(dilate)                                             # invert image colors
# cv2.imshow('image', image)
# cv2.waitKey(0)


Predict Captcha

In [ ]:
for i in range(0, 180, 36):
    # get character from captcha
    letter = image[5:40, i:i+36]
    # resize image to 20x30
    letter = cv2.resize(letter, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))
    # dilate image
    letter = cv2.dilate(letter, np.ones((2, 2), np.uint8), iterations=1)
    # flatten image to 1d numpy array
    letter = letter.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))
    letter = np.float32(letter)
    # get nearest neighbor
    res = kNearest.predict(letter)
    # print predicted character
    print(chr(int(res[0])), end = '')

    # show image
    cv2.rectangle(img, (i, 5), (i+36, 40), (0, 255, 0), 1)
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
cv2.destroyAllWindows()